In [2]:
import ffmpeg, os
from pprint import pprint

def checkIfVideo(path : str)->bool:
    try:
      streams = ffmpeg.probe(path)["streams"]
      for stream in streams:
        # pprint(stream)
        if stream['codec_type'] == "video" : 
            return True
    except:
      return False
    return False

def exportFrames(videoPath : str, outputPath :str, width : int, height:int)->str:
    if not os.path.exists(videoPath):
        raise ValueError('input file path does not exist')
    if not os.path.exists(outputPath):
        raise ValueError('output folder path does not exist')
    if not checkIfVideo(videoPath):
        raise ValueError('input file is not a video')

    print("exporting sequence image from video...")

    probe = ffmpeg.probe(videoPath)
    width = int(probe['streams'][0]['width'])
    nb_frames = int(probe['streams'][0]['nb_frames'])
    duration = float(probe['streams'][0]['duration'])
    
    # pprint(probe['streams'][0])
    sequenceName = os.path.basename(videoPath).split(".")[0]
    outputFolderPath = outputPath + '/' + sequenceName + "_" 
    folderInc = 1
    while(os.path.exists(outputFolderPath + str(folderInc).zfill(3))):
        folderInc+=1
    outputDir = outputFolderPath + str(folderInc).zfill(3) + "/" 
    os.mkdir(outputDir)
    try:
        ffmpeg                                                          \
        .input(videoPath)                    \
        .filter('scale', height,width)                                     \
        .output(outputDir + sequenceName + '_%04d.png').run(capture_stdout=True, capture_stderr=True)
    except ffmpeg.Error as e:
        print('stdout:', e.stdout.decode('utf8'))
        print('stderr:', e.stderr.decode('utf8'))
        raise e
        
    print("Exported sequence in folder " + outputDir)
    return outputDir
# help(ffmpeg.input)


In [83]:
from torch import autocast
import requests
import torch
from PIL import Image
from io import BytesIO
import gc

from diffusers import StableDiffusionImg2ImgPipeline
# C:\Users\goulc\Documents\stablediffusion>python scripts/gradio/depth2img.py configs/stable-diffusion/v2-midas-inference.yaml ckpt\512-depth-ema.ckpt
class Model():
    def __init__(self, model_id_or_path):
        #clear GPU memory if necessary to load the model
        gc.collect()
        torch.cuda.empty_cache()

        # load the pipeline
        self.device = "cuda"
        # model_id_or_path = "nitrosocke/redshift-diffusion"
        self.model_id_or_path = "dallinmackay/Van-Gogh-diffusion"
        self.pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
            model_id_or_path, 
            torch_dtype=torch.float32).to(self.device)
        # or download via git clone https://huggingface.co/CompVis/stable-diffusion-v1-4
        # and pass `model_id_or_path="./stable-diffusion-v1-4"` without having to use `use_auth_token=True`.
    def clearPipe(self):
        self.pipe = 0
        torch.cuda.empty_cache()
        gc.collect()

    def resetPipe(self, newModel = ''):
        if newModel :
            self.model_id_or_path = newModel
        self.pipe = 0
        torch.cuda.empty_cache()
        gc.collect()
        self.pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
            self.model_id_or_path, 
            torch_dtype=torch.float32
        ).to(self.device)

    def createImages(self,sequenceFolder,outputFolder,prompt,negative_prompt,strength=0.35, guidance_scale=8, early= False,width=-1,height=-1):
        if self.pipe == "" :
            raise ValueError('pipe is not set, reset it with resetPipe()')
        try:
            sequenceName = sequenceFolder.split('/')[-1]
            print(sequenceName)
            folderInc = 1
            while(os.path.exists(outputFolder + '/' + sequenceName + str(folderInc).zfill(3))):
                folderInc+=1
            outputFolder = outputFolder + "/" + sequenceName + "/" + sequenceName + str(folderInc).zfill(3) + "/" 
            print(outputFolder)
            os.mkdir(outputFolder)
            for subdir, dirs, files in os.walk(sequenceFolder):
                for file in files:
                    init_image = Image.open(os.path.join(subdir, file)).convert("RGB")
                    originalSize = init_image.size
                    init_image = init_image.resize((768, 512))
                    with autocast("cuda"):
                        images = self.pipe(prompt=prompt, negative_prompt= negative_prompt, init_image=init_image, strength=strength, guidance_scale=guidance_scale).images
                    imageFinale = images[0].resize(originalSize)
                    imageFinale.save(outputFolder + "/" + file )
                    if early :
                        break
        except Exception as ex:
            print(ex)
        





In [78]:
filepath = 'C:/Users/goulc/Documents/stablediffusion/inputs/1088410305-preview.mp4'
otuputfolderpath = "C:/Users/goulc/Documents/stablediffusion/inputs"
sequenceFolder =  exportFrames(filepath,otuputfolderpath, 1920,1080)

exporting sequence image from video...
Exported sequence in folder C:/Users/goulc/Documents/stablediffusion/inputs/1088410305-preview_004/


In [ ]:

try:
    model.clearPipe()
except Exception as ex:
    print(ex)
    
# prompt = "lvngvncnt, one black man with white glasses is playing the drums alone in an empty room, his left arm is lowered, he is sining into a microphone, highly detailed"
# negative_prompt = "wearing a hat"
model = Model("dallinmackay/Van-Gogh-diffusion")
outputFolder = "C:/Users/goulc/Documents/stablediffusion/outputs"
prompt = "lvngvncnt, one white bald man with glasses is playing the saxophone alone in an empty room, his left arm is lowered, he is sining into a microphone, highly detailed"
prompt = "lvngvncnt, a farmer with a clown hat, highly detailed"
negative_prompt = "wearing a hat"
strength=0.4
guidance_scale=7
model.createImages(sequenceFolder,outputFolder,prompt,negative_prompt,strength, guidance_scale, early=False)


# model.resetPipe()

In [54]:
import numpy as np
labels=np.array([[[1,2,3],[1,2,3],[1,2,3]],
                 [[1,2,3],[1,2,3],[1,2,3]]])
print(labels[None].shape)
print(labels[None].transpose(0,3,2,1))
# def clear(model):
#     try:
#         del model.pipe
#     except:
#         pass
#     return

# if 'model' in locals():
#     try:
#         del model.pipe
#     except:
#         pass
# clear.
# torch.cuda.empty_cache()
# gc.collect()
# torch.cuda.empty_cache()
# gc.collect()



(1, 2, 3, 3)
[[[[1 1]
   [1 1]
   [1 1]]

  [[2 2]
   [2 2]
   [2 2]]

  [[3 3]
   [3 3]
   [3 3]]]]


In [18]:
outputFolder = "C:/Users/goulc/Documents/stablediffusion/outputs"
prompt = 'a cat'
negative_prompt = 'a dog'
strength=0.35
guidance_scale=8


                   
model.createImages(sequenceFolder,outputFolder,prompt,negative_prompt,strength, guidance_scale)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\goulc\AppData\Local\Temp\ipykernel_22712\2557690339.py:9 in <module>                    │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ 'C:\\Users\\goulc\\AppData\\Local\\Temp\\ipykernel_22712\\2557690339.py'                         │
│                                                                                                  │
│ C:\Users\goulc\AppData\Local\Temp\ipykernel_22712\865377466.py:34 in createImages                │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ 'C:\\Users\\goulc\\AppData\\Local\\Temp\\ipykernel_22712\\865377466.py'                          │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
FileExistsError: [WinError 183] Cannot create a file when that file already exists: 
'C:/Users/goulc/Documents/stablediffusion/outputs'